# **stanford-crfm/BioMedLM**

# **Step 1: Libraries**

In [1]:
!pip install langchain accelerate bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
import os
from tqdm import tqdm
import pandas as pd
import torch
import accelerate
from langchain import PromptTemplate, HuggingFacePipeline
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

# **Step 2: Load Data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Medical_Transcription_Extraction/mtsamples.csv')

In [5]:
transcription_list = df['transcription'].to_list()

# **Step 3: Model**

In [6]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

**To get the Hugging Face User Access token, click [here](https://huggingface.co/settings/tokens)**

In [21]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large", trust_remote_code=True, max_new_token=1024)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/BioGPT-Large",
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    device_map="auto"
    )

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/566k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

ImportError: You need to install sacremoses to use BioGptTokenizer. See https://pypi.org/project/sacremoses/ for installation.

In [14]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
)

In [15]:
llm = HuggingFacePipeline(pipeline=pipe)

# **Step 4: Inference**

In [16]:
def get_chatglm_response(transcription):
    # prompt = f"""This is a patient's transcription: '{transcription}'
    # For the given patient's transcription, do the next actions sequentially:
    # First, propose the patient's age. If there is no words about age, return unknown.
    # Second, propose no more than ten words that summarize the treatment. If you don't know the treatment, assume '''unknown'''
    # Note: don't use any other words.
    # Then, show your response following the next format:
    # Age: <age>
    # Treatment: The patient will get <treatment> as treatment.
    # """
    prompt = f"This is a patient's transcription: '{transcription}' \n\n Please use this format as the answer: The patient age is [Your extracted answer]. The patient will get [Your extracted answer] as treatment. Note: If the treatment has already been administered, it's also considered as the treatment, the format will be revised slightly: The patient got [Your extracted answer] as treatment. If there is no specific neither age or treatment , you can return unknown, unknown. If there is no specific age but treatment, you can return unknown, The patient will get [Your extracted answer] as treatment. There may be not only one treatment"
    prompt = PromptTemplate.from_template(prompt)
    response = llm.invoke(prompt.format(), max_length=500)
    return response

In [20]:
res = []
for transcription in tqdm(transcription_list[0:2]):
    response = get_chatglm_response(transcription)
    res.append(response)
print(res)

  0%|          | 0/2 [00:00<?, ?it/s]


ValueError: Input length of input_ids is 490, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.